In [272]:
import pandas as pd
import numpy as np
import folium
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup as BS
import re
import warnings
import bokeh
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codecs
from json import JSONDecoder
from functools import partial

from folium import plugins
from folium.plugins import HeatMap
import datetime
import geopy.distance
import mpu
from folium.plugins import MarkerCluster

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
warnings.filterwarnings("ignore")

In [273]:
faults = pd.read_csv('data\\J1939Faults.csv',header=None)


In [274]:
columns1=["FaultId",
"ESS_Id",
"EventTimeStamp",
"eventDescription",
"actionDescription",
"ecuSoftwareVersion",
"ecuSerialNumber",
"ecuModel",
"ecuMake",
"ecuSource",
"spn",
"fmi",
"active",
"activeTransitionCount",
"faultValue",
"EquipmentID",
"MCTNumber",
"Latitude",
"Longitude",
"LocationTimeStamp"] 

In [275]:
faults.columns=columns1

In [276]:
faults.head()

,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [277]:
faults.shape

(1085683, 20)

In [278]:
bk=faults.groupby("FaultId").agg({"FaultId": [np.size]})
bk.head()

,FaultId
,size
FaultId,
1,1
2,1
3,1
4,1
5,1


In [279]:
list(bk)

[('FaultId', 'size')]

In [280]:
#bk["FaultId"].sort_values("size").head()

In [281]:
diagnostic = pd.read_csv('data\\VehicleDiagnosticOnboardData.csv',header=None)

In [282]:
diagnostic.head()

,0,1,2,3
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [283]:
columns2=["Id","Name","Value","FaultId"]

diagnostic.columns=columns2

diagnostic["Value"].shape

(11687087,)

In [284]:
diagnostic.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [285]:
merged=pd.merge(diagnostic,faults,on="FaultId",how="inner")

In [286]:
merged.shape

(11686998, 23)

In [287]:
derated=merged.loc[(((merged["spn"]==5246) & (merged["fmi"]==0)) | ((merged["spn"]==5246) & (merged["fmi"]==15)) | ((merged["spn"]==5246) & (merged["fmi"]==16))),:]

In [288]:
derated.shape

(9033, 23)

In [289]:
derated=derated.drop_duplicates(subset=['EquipmentID', 'EventTimeStamp'], keep="first")

In [290]:
derated.head(5)

,Id,Name,Value,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
418,419,IgnStatus,True,46,990931,2015-02-21 12:10:51.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27.000
16232,16233,IgnStatus,True,1919,1007751,2015-02-22 19:44:55.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27.000
17486,17487,LampStatus,1023,2059,1010486,2015-02-23 04:00:21.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,False,1,NaN,1395,105349612,36.066666,-86.434537,2015-02-23 01:06:06.000
17735,17736,IgnStatus,False,2090,1011009,2015-02-23 05:05:44.000,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000
25573,25574,IgnStatus,True,2972,1026305,2015-02-23 15:54:22.000,NaN,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,NaN,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58.000


In [291]:
derated["ecuMake"].value_counts()

CMMNS      524
unknown    274
?MMNS        1
Name: ecuMake, dtype: int64

In [292]:
bk=derated.groupby(["Latitude","Longitude"])["Latitude","Longitude"].agg({np.size})
bk=bk["Latitude"].sort_values(["size"])
bk.tail(100)

,,size
Latitude,Longitude,
35.997500,-86.595092,1.0
35.587592,-86.444027,1.0
35.936666,-87.189768,1.0
35.936620,-87.189768,1.0
36.007916,-86.427546,1.0
36.009953,-83.523564,1.0
35.587777,-86.444074,1.0
35.494722,-86.427916,1.0
35.523935,-84.591250,1.0


In [293]:
service = pd.read_excel('data\\Service Fault Codes_1_0_0_167.xlsx')

In [294]:
service.head()

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


In [295]:

service.groupby(["SPN","J1939 FMI"]).size().reset_index()

,SPN,J1939 FMI,0
0,22,3,1
1,22,4,1
2,27,2,1
3,27,3,1
4,27,4,1
5,27,13,1
6,27,16,1
7,29,2,1
8,38,2,1
9,38,3,1


In [296]:
service["SPN"].value_counts()

524286    765
524287    585
611        86
3464       27
102        22
1761       21
105        21
110        21
7581       21
633        21
641        19
1172       18
190        18
3226       18
3216       18
1127       17
1244       17
3364       17
91         17
157        16
8289       16
2791       16
3465       16
159        15
94         15
629        15
103        15
1075       15
3031       15
3470       15
3251       15
1209       14
84         14
3361       14
1292       14
1176       14
51         14
3227       14
3217       14
4363       14
175        14
1188       13
111        13
632        13
4360       13
4334       13
639        13
4765       13
171        12
3246       12
100        12
1231       11
649        11
108        11
97         11
5848       11
1390       11
3057       11
724        11
5401       11
3242       11
5749       11
3249       11
8322       11
168        10
1300       10
132        10
1301       10
521209     10
1131       10
3265       10
1325  

In [297]:
service["Lamp Color"].value_counts()

Amber          4971
Red             881
None            871
Maintenance     401
Name: Lamp Color, dtype: int64

In [298]:
service["J1939 FMI"].value_counts()

31    2331
3      754
4      730
2      426
16     372
18     267
0      233
15     216
9      211
11     191
5      182
7      179
14     143
17     138
13     133
1      120
12     108
10     104
6       86
19      77
20      61
21      49
8       13
Name: J1939 FMI, dtype: int64

In [299]:
output = service.pivot_table(index='SPN', columns='J1939 FMI', values='Cummins Fault Code')

In [300]:
output.head(5)

J1939 FMI,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,31
SPN,,,,,,,,,,,,,,,,,,,,,,,
22,NaN,NaN,NaN,719.0,729.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,1228.0,2271.0,2272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2348.0,NaN,NaN,1625.0,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,NaN,NaN,5319.0,5317.0,5318.0,NaN,NaN,NaN,NaN,6342.0,NaN,NaN,NaN,6343.0,NaN,NaN,NaN,NaN,1439.0,5328.0,NaN,NaN,NaN
46,NaN,NaN,218.0,216.0,217.0,774.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,775.0,776.0,NaN,NaN,NaN,NaN


In [303]:
lat=38
long=-84

map=folium.Map(location=[lat,long],zoom_start=2)

map2=folium.Map(location=[lat,long],zoom_start=2)
map3=folium.Map(location=[lat,long],zoom_start=2)

In [304]:
for row in derated.iterrows():
    row_values = row[1]
    location = [row_values['Latitude'], row_values['Longitude']]
    marker = folium.CircleMarker(location=location,radius=1)
    marker.add_to(map)
map